# Práctica 1: Datos personales y anonimización

**Utilizando los datos recogidos en el fichero de datos abiertos de PDI de la UAM correspondiente al año 2022, realizar, para cada campo del fichero exceptuando aquellos cuyo nombre empieza por "cod_", "lat_" y "lon_", lo siguiente:**

* **Inferir el valor más probable de cada campo para el caso del profesor Ortigosa.**

* **Calcular el nivel de certeza (%) de que cada valor inferido corersponda realmente al profesor Ortigosa.**

* **Una breve descripción de infrencia utilizado en cada caso.**

#### Datos de partida y condiciones
Aseguramos que:
* El profesor Ortigosa estaba en 2022 en la UAM (aparece en el fichero).
* El profesor Ortigosa era de género masculino en 2022.
* El profesor Ortigosa pertenecía al Departamento de Ingeniería Informática en 2022.
* Puede utilizarse para el ejercicio cualquier otra información que se pueda obtener por cualquier vía adicional, pero debe especificarse claramente en la respuesta tanto la información usada como su origen.

## Preprocesado de los datos

In [1]:
import pandas as pd
import os
from IPython.core.display import display, HTML

C:\Users\imcir\AppData\Local\Temp\ipykernel_38796\2646090995.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [2]:
csv_name = "uam-personal-pdi-2022-anonimizado.csv"
df = pd.read_csv(os.path.join(os.curdir, csv_name))

# Eliminamos las columnas que indica el enunciado que no se deben utilizar
df = df.loc[:, ~df.columns.str.startswith(("cod_", "lat_", "lon_"))]

In [3]:
# Definimos esta función para hacer inferencia estadística sobre los datos
def valores_unicos(df: pd.DataFrame, max_alto='400px', max_ancho='100%'):
    resultados = []
    for columna in df.columns:
        valores_unicos = df[columna].value_counts(dropna=False)
        total = len(df[columna])
        for valor, cantidad in valores_unicos.items():
            porcentaje = (cantidad / total) * 100
            resultados.append([columna, valor, cantidad, porcentaje])
    df_resultado = pd.DataFrame(resultados, columns=['Columna', 'Valor Único', 'Recuento', 'Porcentaje'])
    
    # Mostrar como tabla scrolleable
    estilo = f"overflow:auto; height:{max_alto}; width:{max_ancho};"
    html = f'<div style="{estilo}">{df_resultado.to_html(index=False)}</div>'
    display(HTML(html))

## Variables pivote

In [4]:
# Utilizamos las variables pivote (Género y Unidad responsable) para realizar un filtrado inicial de los datos
# Eliminamos también las columnas que no nos aportan información (IMPORTANTE)

df_filtered = df.drop(columns=['IMPORTANTE'])

df_var_piv = df_filtered[(df_filtered['des_genero'] == "Hombre") & \
                         (df_filtered['des_unidad_responsable'] == "Departamento de Ingeniería Informática")]

valores_unicos(df_var_piv)

Columna,Valor Único,Recuento,Porcentaje
des_universidad,Universidad Autónoma de Madrid,56,100.000000
anio,2022,56,100.000000
des_pais_nacionalidad,España,52,92.857143
des_pais_nacionalidad,Italia,2,3.571429
des_pais_nacionalidad,Irán (República Islámica del),1,1.785714
des_pais_nacionalidad,Irlanda,1,1.785714
des_continente_nacionalidad,Europa,55,98.214286
des_continente_nacionalidad,Asia,1,1.785714
des_agregacion_paises_nacionalidad,Europa meridional,54,96.428571
des_agregacion_paises_nacionalidad,Asia meridional,1,1.785714


## Bloque 1

In [5]:
# El bloque 1 (Universidad y año del conjunto de datos:

df_blq1 = df_var_piv[['des_universidad',\
                      'anio']]

valores_unicos(df_blq1)

Columna,Valor Único,Recuento,Porcentaje
des_universidad,Universidad Autónoma de Madrid,56,100.0
anio,2022,56,100.0


#### Inferencia estadística a partir de los datos del bloque 1

Este bloque no nos aporta información relevante sobre nuestro objetivo. Solamente indica la universidad a la que hacen referencia los datos (Universidad Autónoma de Madrid) y el año natural al que hacen referencia los datos (2022).

## Bloque 2

In [6]:
# El bloque 2 (Nacionalidad):

df_blq2 = df_var_piv[['des_pais_nacionalidad',\
                      'des_continente_nacionalidad',\
                      'des_agregacion_paises_nacionalidad']]

valores_unicos(df_blq2)

Columna,Valor Único,Recuento,Porcentaje
des_pais_nacionalidad,España,52,92.857143
des_pais_nacionalidad,Italia,2,3.571429
des_pais_nacionalidad,Irán (República Islámica del),1,1.785714
des_pais_nacionalidad,Irlanda,1,1.785714
des_continente_nacionalidad,Europa,55,98.214286
des_continente_nacionalidad,Asia,1,1.785714
des_agregacion_paises_nacionalidad,Europa meridional,54,96.428571
des_agregacion_paises_nacionalidad,Asia meridional,1,1.785714
des_agregacion_paises_nacionalidad,Europa septentrional,1,1.785714


#### Inferencia estadística a partir de los datos del bloque 2

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `des_pais_nacionalidad`: es el nombre del país correspondiente a la nacionalidad del empleado.
* `des_continente_nacionalidad`: es el nombre del continente correspondiente a la nacionalidad del empleado.
* `des_agregacion_paises_nacionalidad`: es el nombre de la agregación de países correspondiente a la nacionalidad del empleado.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables. Sin embargo, vamos a inferir la probabilidad de cada variable:
* `des_pais_nacionalidad`: España con 92.86% de seguridad, Italia con 3.57% de seguridad, Irán e Irlanda con 1.79% de seguridad cada uno.
* `des_continente_nacionalidad`: Europa con 98.21% de seguridad y Asia con 1.79% de seguridad.
* `des_agregacion_paises_nacionalidad`: Europa meridional con 96.43% de seguridad, Europa septentrional con 1.79% de seguridad y Asia meridional con 1.79% de seguridad.

## Bloque 3

In [7]:
# El bloque 3 (Residencia):

df_blq3 = df_var_piv[['des_comunidad_residencia',\
                      'des_provincia_residencia',\
                      'des_municipio_residencia']]

valores_unicos(df_blq3)

Columna,Valor Único,Recuento,Porcentaje
des_comunidad_residencia,Madrid,55,98.214286
des_comunidad_residencia,Valencia,1,1.785714
des_provincia_residencia,Madrid,55,98.214286
des_provincia_residencia,Valencia/València,1,1.785714
des_municipio_residencia,MADRID,25,44.642857
des_municipio_residencia,COLMENAR VIEJO,6,10.714286
des_municipio_residencia,ALCOBENDAS,4,7.142857
des_municipio_residencia,SAN SEBASTIÁN DE LOS REYES,4,7.142857
des_municipio_residencia,TRES CANTOS,3,5.357143
des_municipio_residencia,"ROZAS DE MADRID, LAS",1,1.785714


#### Inferencia estadística a partir de los datos del bloque 3

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `des_comunidad_residencia`: es el nombre la comunidad autónoma del domicilio del empleado.
* `des_provincia_residencia`: es el nombre de la provincia del domicilio del empleado.
* `des_municipio_residencia`: es el nombre del municipio del domicilio del empleado.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables. Sin embargo, vamos a inferir la probabilidad de cada variable:
* `des_comunidad_residencia`: Madrid con 98.21% de seguridad y Valencia con 1.79% de seguridad.
* `des_provincia_residencia`: Madrid con 98.21% de seguridad y Valencia con 1.79% de seguridad.
* `des_municipio_residencia`: Madrid con 44.64% de seguridad, Colmenar Viejo con 10.71% de seguridad, Alcobendas y San Sebastián de los Reyes con 7.14% de seguridad cada uno, Tres Cantos con 5.35% de seguridad y Las Rozas de Madrid, Colmenarejo, Alcalá de Henares, San Fernando de Henares, Pozuelo de Alarcón, Fuenlabrada, Hoyo de Manzanares, El Bolao, Valencia, Moralzarzal, Manzanares el Real, Galapagar, Algete y Alcorcón con 1.79% de seguridad cada uno. 

## Bloque 4

In [8]:
# El bloque 4 (Año nacimiento):

df_blq4 = df_var_piv[['anio_nacimiento']]

valores_unicos(df_blq4)

Columna,Valor Único,Recuento,Porcentaje
anio_nacimiento,1982,4,7.142857
anio_nacimiento,1965,4,7.142857
anio_nacimiento,1967,3,5.357143
anio_nacimiento,1964,3,5.357143
anio_nacimiento,1976,3,5.357143
anio_nacimiento,1978,3,5.357143
anio_nacimiento,1986,3,5.357143
anio_nacimiento,1968,2,3.571429
anio_nacimiento,1969,2,3.571429
anio_nacimiento,1966,2,3.571429


#### Inferencia estadística a partir de los datos del bloque 4

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `anio_nacimiento`: es el año de nacimiento del empleado.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables (y tampoco serviría de nada puesto que se permutan los datos). Sin embargo, vamos a inferir la probabilidad de cada variable:
* `anio_nacimiento`: 1965 y 1982 con 7.14% de seguridad cada uno, 1967, 1964, 1976, 1978 y 1986 con 5.36% de seguridad cada uno, 1968, 1969, 1966, 1977, 1984, 1974, 1975, 1963 y 1962 con 3.57% de seguridad cada uno y 1994, 1961, 1988, 1958, 1972, 1960, 1980, 1973, 1989, 1985, 1955, 1997, 1983, 1981, 1995 con 1.79% de seguridad cada uno.

Como esta información tampoco nos aporta gran cosa, podemos inferir las probabilidades de que haya nacido en cada década:
* 1950s: 3.57% de seguridad.
* 1960s: 39.28% de seguridad.
* 1970s: 25.00% de seguridad.
* 1980s: 26.79% de seguridad.
* 1990s: 5.34% de seguridad.


## Bloque 5

In [9]:
# El bloque 5 (Contrato actual):

df_blq5 = df_var_piv[['des_tipo_personal',\
                      'des_categoria_cuerpo_escala',\
                      'des_tipo_contrato',\
                      'des_dedicacion',\
                      'num_horas_semanales_tiempo_parcial',\
                      'des_situacion_administrativa']]

df_blq5_nan = df_blq5[((df_blq5['des_categoria_cuerpo_escala'] == "Profesor Titular de Universidad") | (df_blq5['des_categoria_cuerpo_escala'].isna()))]

valores_unicos(df_blq5_nan)

Columna,Valor Único,Recuento,Porcentaje
des_tipo_personal,Funcionario de carrera,17,100.000000
des_categoria_cuerpo_escala,Profesor Titular de Universidad,17,100.000000
des_tipo_contrato,Personal Funcionario\Personal eventual,17,100.000000
des_dedicacion,Dedicación a Tiempo Completo,16,94.117647
des_dedicacion,Dedicación a Tiempo Parcial,1,5.882353
num_horas_semanales_tiempo_parcial,NaN,16,94.117647
num_horas_semanales_tiempo_parcial,12.0,1,5.882353
des_situacion_administrativa,Servicio Activo,17,100.000000


#### Inferencia estadística a partir de los datos del bloque 5

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable y vamos a citar las fuentes de donde hemos obtenido la información filtrada:
* `des_tipo_personal`: es la descripción del tipo de personal del empleado.
* `des_categoria_cuerpo_escala`: es la descripción de la categoría/cuerpo/escala del empleado. Obtenido de **Portal Científico de la UAM**.
* `des_tipo_contrato`: es la descripción del tipo de contrato del empleado.
* `des_dedicacion`: es la descripción del régimen de dedicación del empleado.
* `num_horas_semanales_tiempo_parcial`: es el número de horas semanales en contrato del profesorado a tiempo parcial.
* `des_situacion_administrativa`: es la descripción de la situación administrativa del empleado.

Para continuar, vamos a ver qué información podemos obtener de este bloque a partir de la información de fuentes externas.

Considerando o no los valores `NaN` se obtienen los mismos resultados:
* `des_categoria_cuerpo_escala`: Profesor Titular de Universidad con 100% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `des_tipo_personal`: Funcionario de carrera con 100% de seguridad.
* `des_tipo_contrato`: Personal Funcionario\Personal eventual con 100% de seguridad.
* `des_dedicacion`: Dedicación a Tiempo Completo con 94.12% de seguridad y Dedicación a Tiempo parcial con 5.88% de seguridad.
* `num_horas_semanales_tiempo_parcial`: 12 horas con 100% de seguridad si tiene Dedicación a Tiempo parcial, sino esta categoría contiene `NaN`.
* `des_situacion_administrativa`: Servicio Activo con 100% de seguridad.

In [10]:
# El bloque 6 (Cargo remunerado):

df_blq6 = df_var_piv[['ind_cargo_remunerado']]

valores_unicos(df_blq6)

Columna,Valor Único,Recuento,Porcentaje
ind_cargo_remunerado,N,45,80.357143
ind_cargo_remunerado,S,11,19.642857


#### Inferencia estadística a partir de los datos del bloque 6

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `ind_cargo_remunerado`: es el indicador de si ocupa algún cargo unipersonal remunerado económicamente en la Universidad.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables (y tampoco serviría de nada puesto que se permutan los datos). Sin embargo, vamos a inferir la probabilidad de cada variable:
* `ind_cargo_remunerado`: N (No) con 80.36% de seguridad y S (Sí) con 19.64% de seguridad.

## Bloque 7

In [11]:
# El bloque 7 (Doctorado):

df_blq7 = df_var_piv[['des_titulo_doctorado',\
                      'des_pais_doctorado',\
                      'des_continente_doctorado',\
                      'des_agregacion_paises_doctorado',\
                      'des_universidad_doctorado',\
                      'anio_lectura_tesis',\
                      'anio_expedicion_titulo_doctor',\
                      'des_mencion_europea']]

df_blq7_nan = df_blq7[((df_blq7['des_titulo_doctorado'] == "Uno") | (df_blq7['des_titulo_doctorado'].isna())) & \
                      ((df_blq7['des_pais_doctorado'] == "España") | (df_blq7['des_pais_doctorado'].isna())) & \
                      ((df_blq7['des_continente_doctorado'] == "Europa") | (df_blq7['des_continente_doctorado'].isna())) & \
                      ((df_blq7['des_agregacion_paises_doctorado'] == "Europa meridional") | (df_blq7['des_agregacion_paises_doctorado'].isna())) & \
                      ((df_blq7['des_universidad_doctorado'] == "Universidad Autónoma de Madrid") | (df_blq7['des_universidad_doctorado'].isna())) & \
                      ((df_blq7['anio_lectura_tesis'] == 2000) | (df_blq7['anio_lectura_tesis'].isna()))]

valores_unicos(df_blq7_nan)

df_blq7 = df_blq7[(df_blq7['des_titulo_doctorado'] == "Uno") & \
                  (df_blq7['des_pais_doctorado'] == "España") & \
                  (df_blq7['des_continente_doctorado'] == "Europa") & \
                  (df_blq7['des_agregacion_paises_doctorado'] == "Europa meridional") & \
                  (df_blq7['des_universidad_doctorado'] == "Universidad Autónoma de Madrid") & \
                  (df_blq7['anio_lectura_tesis'] == 2000)]

valores_unicos(df_blq7)

Columna,Valor Único,Recuento,Porcentaje
des_titulo_doctorado,Uno,6,100.000000
des_pais_doctorado,NaN,4,66.666667
des_pais_doctorado,España,2,33.333333
des_continente_doctorado,NaN,4,66.666667
des_continente_doctorado,Europa,2,33.333333
des_agregacion_paises_doctorado,NaN,4,66.666667
des_agregacion_paises_doctorado,Europa meridional,2,33.333333
des_universidad_doctorado,NaN,4,66.666667
des_universidad_doctorado,Universidad Autónoma de Madrid,2,33.333333
anio_lectura_tesis,NaN,4,66.666667


Columna,Valor Único,Recuento,Porcentaje
des_titulo_doctorado,Uno,1,100.0
des_pais_doctorado,España,1,100.0
des_continente_doctorado,Europa,1,100.0
des_agregacion_paises_doctorado,Europa meridional,1,100.0
des_universidad_doctorado,Universidad Autónoma de Madrid,1,100.0
anio_lectura_tesis,2000.0,1,100.0
anio_expedicion_titulo_doctor,2000.0,1,100.0
des_mencion_europea,No,1,100.0


#### Inferencia estadística a partir de los datos del bloque 7

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable y vamos a citar las fuentes de donde hemos obtenido la información filtrada:
* `des_titulo_doctorado`: es la descripción de si el empleado tiene título de doctor, y cuantía. Obtenido de **Portal Científico de la UAM** y **UAM Biblioteca**.
* `des_pais_doctorado`: es el nombre del país en el que se ha obtenido el título de doctorado. Obtenido de **Portal Científico de la UAM** y **UAM Biblioteca**.
* `des_continente_doctorado`: es el nombre del continente correspondiente al país en el que ha obtenido el título de Doctorado. Obtenido de **Portal Científico de la UAM** y **UAM Biblioteca**.
* `des_agregacion_paises_doctorado`: es el nombre de la agregación de países correspondiente al país en el que ha obtenido el título de Doctorado. Obtenido de **Portal Científico de la UAM** y **UAM Biblioteca**.
* `des_universidad_doctorado`: es el nombre de la Universidad en la que ha obtenido el título de Doctorado, o clasificación si no es española.
* `anio_lectura_tesis`: es el año de lectura de la tesis, si procede. Obtenido de **Portal Científico de la UAM** y **UAM Biblioteca**.
* `anio_expedicion_titulo_doctor`: es el año de expedición del título de doctor, si procede. No se ha obtenido información de fuentes externas.
* `des_mencion_europea`: es la descripción de si tiene mención europea o internacional en alguno de los títulos de doctorado. No se ha obtenido información de fuentes externas.

Para continuar, vamos a ver qué información podemos obtener de este bloque a partir de la información de fuentes externas.

Si consideramos los valores `NaN` tenemos que:
* `des_titulo_doctorado`: Uno con 100% de seguridad.
* `des_pais_doctorado`: España con 33.33% de seguridad.
* `des_continente_doctorado`: Europa con 33.33% de seguridad.
* `des_agregacion_paises_doctorado`: Europa meridional con 33.33% de seguridad.
* `des_universidad_doctorado`: Universidad Autónoma de Madrid con 33.33% de seguridad.
* `anio_lectura_tesis`: 2000 con 33.33% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `anio_expedicion_titulo_doctor`: 2000 con un 16.67% de seguridad, 2010 con un 16.67% de seguridad y 2014 con un 16.67% de seguridad.
* `des_mencion_europea`: No con un 100% de seguridad.

Si no consideramos los valores `NaN` tenemos que ya que suponemos que los datos encontrados mediante fuentes externas están reflejados en los datos originales:
* `des_titulo_doctorado`:  Uno con 100% de seguridad.
* `des_pais_doctorado`: España con 100% de seguridad.
* `des_continente_doctorado`: Europa con 100% de seguridad.
* `des_agregacion_paises_doctorado`: Europa meridional con 100% de seguridad.
* `des_universidad_doctorado`: Universidad Autónoma de Madrid con 100% de seguridad.
* `anio_lectura_tesis`: 2000 con 100% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `anio_expedicion_titulo_doctor`: 2000 con un 100% de seguridad.
* `des_mencion_europea`: No con un 100% de seguridad.

In [12]:
# El bloque 8 (Departamento):

df_blq8 = df_var_piv[['des_tipo_unidad_responsable',\
                      'des_unidad_responsable',\
                      'des_area_conocimiento']]

df_blq8_nan = df_blq8[((df_blq8['des_tipo_unidad_responsable'] == "Departamento") | (df_blq8['des_tipo_unidad_responsable'].isna())) & \
                      ((df_blq8['des_area_conocimiento'] == "Lenguajes y Sistemas Informáticos") | (df_blq8['des_area_conocimiento'].isna()))]

valores_unicos(df_blq8_nan)

Columna,Valor Único,Recuento,Porcentaje
des_tipo_unidad_responsable,Departamento,23,100.0
des_unidad_responsable,Departamento de Ingeniería Informática,23,100.0
des_area_conocimiento,Lenguajes y Sistemas Informáticos,23,100.0


#### Inferencia estadística a partir de los datos del bloque 8

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable y vamos a citar las fuentes de donde hemos obtenido la información filtrada:
* `des_tipo_unidad_responsable`: es la descripción del tipo de unidad en la que está adscrito el PDI.
* `des_unidad_responsable`: es la descripción de la unidad responsable. Obtenido de **Enunciado**.
* `des_area_conocimiento`: es la descripción del área de conocimiento. Obtenido de **Portal Científico de la UAM**.

Para continuar, vamos a ver qué información podemos obtener de este bloque a partir de la información de fuentes externas.

Considerando o no los valores `NaN` se obtienen los mismos resultados:
* `des_unidad_responsable`: Departamento de Ingeniería Informática con 100% de seguridad.
* `des_area_conocimiento`: Lenguajes y Sistemas Informáticos con 100% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `des_tipo_unidad_responsable`: Departamento con 100% de seguridad.

## Bloque 9

In [13]:
# El bloque 9 (Cuerpo docente e investigador): 

df_blq9 = df_var_piv[['anio_incorporacion_ap',\
                      'anio_incorpora_cuerpo_docente',\
                      'num_trienios',\
                      'num_quinquenios',\
                      'num_sexenios']]

df_blq9_nan = df_blq9[((df_blq9['anio_incorporacion_ap'] >= 2000) | (df_blq9['anio_incorporacion_ap'].isna())) & \
                      (df_blq9['num_sexenios'] == 3) | (df_blq9['num_sexenios'].isna())]

valores_unicos(df_blq9_nan)

df_blq9 = df_blq9[(df_blq9['anio_incorporacion_ap'] >= 2000) & \
                  (df_blq9['num_sexenios'] == 3)]

valores_unicos(df_blq9)

Columna,Valor Único,Recuento,Porcentaje
anio_incorporacion_ap,2001.0,3,60.0
anio_incorporacion_ap,NaN,1,20.0
anio_incorporacion_ap,2002.0,1,20.0
anio_incorpora_cuerpo_docente,2021.0,2,40.0
anio_incorpora_cuerpo_docente,2020.0,1,20.0
anio_incorpora_cuerpo_docente,NaN,1,20.0
anio_incorpora_cuerpo_docente,2022.0,1,20.0
num_trienios,7.0,3,60.0
num_trienios,5.0,1,20.0
num_trienios,6.0,1,20.0


Columna,Valor Único,Recuento,Porcentaje
anio_incorporacion_ap,2001.0,3,75.0
anio_incorporacion_ap,2002.0,1,25.0
anio_incorpora_cuerpo_docente,2021.0,2,50.0
anio_incorpora_cuerpo_docente,2020.0,1,25.0
anio_incorpora_cuerpo_docente,2022.0,1,25.0
num_trienios,7.0,2,50.0
num_trienios,5.0,1,25.0
num_trienios,6.0,1,25.0
num_quinquenios,4.0,2,50.0
num_quinquenios,2.0,1,25.0


#### Inferencia estadística a partir de los datos del bloque 9

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable y vamos a citar las fuentes de donde hemos obtenido la información filtrada:
* `anio_incorporacion_ap`: es el año en el que el PDI funcionario de carrera se incorporó por primera vez a la Administración Pública . Obtenido de **Portal Científico de la UAM**.
* `anio_incorpora_cuerpo_docente`: es el año de incorporación al cuerpo docente universitario como funcionario de carrera.
* `num_trienios`: es el número de trienios (méritos de antigüedad adquiridos en la administración.) a fecha 31 de diciembre.
* `num_quinquenios`: es el número de quinquenios estatales a fecha 31 de diciembre.
* `num_sexenios`: es el número de sexenios estatales a fecha 31 de diciembre. Obtenido de **Portal Científico de la UAM**.

Para continuar, vamos a ver qué información podemos obtener de este bloque a partir de la información de fuentes externas.

Si consideramos los valores `NaN` tenemos que:
* `anio_incorporacion_ap`: 2001 con 60% de seguridad y 2002 con 20% de seguridad.
* `num_sexenios`: 3 con 100% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `anio_incorpora_cuerpo_docente`: 2020 con 20% de seguridad, 2021 con 40% de seguridad y 2022 con 20% de seguridad.
* `num_trienios`: 5 con 20% de seguridad, 6 con 20% de seguridad y 7 con 60% de seguridad.
* `num_quinquenios`: 2 con 20% de seguridad, 3 con 20% de seguridad y 4 con 60% de seguridad.

Si no consideramos los valores `NaN` tenemos que ya que suponemos que los datos encontrados mediante fuentes externas están reflejados en los datos originales:
* `anio_incorporacion_ap`: 2001 con 75% de seguridad y 2002 con 25% de seguridad.
* `num_sexenios`: 3 con 100% de seguridad.

De variables que no hemos logrado obtener información de otras fuentes:
* `anio_incorpora_cuerpo_docente`: 2020 con 25% de seguridad, 2021 con 50% de seguridad y 2022 con 25% de seguridad.
* `num_trienios`: 5 con 25% de seguridad, 6 con 25% de seguridad y 7 con 50% de seguridad.
* `num_quinquenios`: 2 con 25% de seguridad, 3 con 25% de seguridad y 4 con 50% de seguridad.

## Bloque 10

In [14]:
# El bloque 10 (Número de tesis): 

df_blq10 = df_var_piv[['num_tesis']]

valores_unicos(df_blq10)

Columna,Valor Único,Recuento,Porcentaje
num_tesis,NaN,53,94.642857
num_tesis,1.0,3,5.357143


#### Inferencia estadística a partir de los datos del bloque 10

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `num_tesis`: es el número de tesis doctorales dirigidas o codirigidas por el profesor que hayan sido leídas a lo largo del año.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables (y tampoco serviría de nada puesto que se permutan los datos). Sin embargo, vamos a inferir la probabilidad de cada variable:
* `num_tesis`: 1 con 5.36% de seguridad.

Esta información no nos aporta prácticamene información ya que la gran mayoría de resultados son `NaN`.

## Bloque 11

In [15]:
# El bloque 11 (Investigador principal): 

df_blq11 = df_var_piv[['ind_investigador_principal']]

valores_unicos(df_blq11)

Columna,Valor Único,Recuento,Porcentaje
ind_investigador_principal,N,45,80.357143
ind_investigador_principal,S,11,19.642857


#### Inferencia estadística a partir de los datos del bloque 11

Vamos a estudiar la información que se puede inferir de este bloque. Para empezar, vamos a entender qué significa cada variable:
* `ind_investigador_principal`: es el indicador de si ha sido Investigador Principal (IP) de algún proyecto/contrato de investigación durante el año.

Para continuar, vamos a ver qué información podemos obtener de este bloque. No se ha podido obtener información de fuentes externas acerca de estas variables (y tampoco serviría de nada puesto que se permutan los datos). Sin embargo, vamos a inferir la probabilidad de cada variable:
* `ind_investigador_principal`: N (No) con 80.36% de seguridad y S (Sí) con 19.64% de seguridad.

## Resumen

En resumen, hemos podido obtener información de gran relevancia acerca del profesor Álvaro Manuel Ortigosa Juárez, demostrando que a partir de un dataset anonimizado se puede extraer información muy relevante. Además cabe destacar que hemos sido muy conservadores y toda la información utilizada ha sido contrastada con funtes externas fiables. Vamos a mostrar el valor más probable de cada campo para el caso del profesor Ortigosa y su nivel de certeza con que realmente correspondan al profesor Ortigosa:

| Campo                                                        | Valor estimado                                         | Certeza  |
|--------------------------------------------------------------|--------------------------------------------------------|----------|
| Género (`des_genero`)                                         | Hombre                                                 | 100%     |
| País de nacionalidad (`des_pais_nacionalidad`)                | España                                                 | 92.86%   |
| Continente de nacionalidad (`des_continente_nacionalidad`)    | Europa                                                 | 98.21%   |
| Agregación de países de nacionalidad (`des_agregacion_paises_nacionalidad`) | Europa meridional                                      | 96.43%   |
| Comunidad autónoma de residencia (`des_comunidad_residencia`) | Madrid                                                 | 98.21%   |
| Provincia de residencia (`des_provincia_residencia`)          | Madrid                                                 | 98.21%   |
| Municipio de residencia (`des_municipio_residencia`)          | Madrid                                                 | 44.64%   |
| Año de nacimiento (`anio_nacimiento`)                         | 1982 o 1965                                            | 7.14%    |
| Tipo de personal (`des_tipo_personal`)                        | Funcionario de carrera                                 | 100%     |
| Categoría/cuerpo/escala del empleado (`des_categoria_cuerpo_escala`) | Profesor Titular de Universidad                        | 100%     |
| Tipo de contrato (`des_tipo_contrato`)                        | Personal Funcionario/Personal eventual                 | 100%     |
| Régimen de dedicación (`des_dedicacion`)                      | Dedicación a Tiempo Completo                           | 94.12%   |
| Número de horas semanales a tiempo parcial (`num_horas_semanales_tiempo_parcial`) | `NaN`                                                  | 94.12%   |
| Situación administrativa (`des_situacion_administrativa`)     | Servicio Activo                                        | 100%     |
| Cargo remunerado (`ind_cargo_remunerado`)                    | No                                                     | 80.36%   |
| Título de doctorado (`des_titulo_doctorado`)                  | Uno                                                    | 100%     |
| País de doctorado (`des_pais_doctorado`)                      | España                                                 | 100%     |
| Continente de doctorado (`des_continente_doctorado`)          | Europa                                                 | 100%     |
| Agregación de países de doctorado (`des_agregacion_paises_doctorado`) | Europa meridional                                      | 100%     |
| Universidad de doctorado (`des_universidad_doctorado`)        | Universidad Autónoma de Madrid                         | 100%     |
| Año de lectura de la tesis (`anio_lectura_tesis`)             | 2000                                                   | 100%     |
| Año de expedición del título de doctor (`anio_expedicion_titulo_doctor`) | `NaN`                                                  | 50%      |
| Mención europea (`des_mencion_europea`)                       | No                                                     | 100%     |
| Tipo unidad responsable (`des_tipo_unidad_responsable`)       | Departamento                                           | 100%     |
| Unidad responsable (`des_unidad_responsable`)                 | Departamento de Ingeniería Informática                  | 100%     |
| Área de conocimiento (`des_area_conocimiento`)               | Lenguajes y Sistemas Informáticos                       | 100%     |
| Año incorporación Administración Pública (`anio_incorporacion_ap`) | 2001                                                   | 60%      |
| Año incorporación cuerpo docente (`anio_incorpora_cuerpo_docente`) | 2021                                                   | 40%      |
| Número de trienios (`num_trienios`)                           | 7                                                      | 60%      |
| Número de quinquenios (`num_quinquenios`)                     | 4                                                      | 60%      |
| Número de sexenios (`num_sexenios`)                           | 3                                                      | 100%     |
| Número de tesis (`num_tesis`)                                 | `NaN`                                                  | 94.64%   |
| Investigador principal (`ind_investigador_principal`)         | No                                                     | 80.36%   |

Las fuentes externas utilizadas pueden encontrarse en los siguientes url:

https://portalcientifico.uam.es/es/ipublic/researcher/261195

https://repositorio.uam.es/handle/10486/726?locale-attribute=es